In [4]:
import copy
from itertools import product
from classification_refined import classify
from processing import *
from readfiles import *
from decision import filter_summary
from subgraphclass import make_solver_summary
from sklearn.model_selection import train_test_split
from scipy.stats import ttest_ind
from readfiles import corresp_label_file
from graphclass import BrainGraph
from paramopt import graph_options

In [23]:
num = 84  # number of nodes in the graph
tri = int(num * (num + 1) * 0.5)  # we want only the upper diagonal parts since everything below diagonal is 0
big5 = ['NEOFAC_A', 'NEOFAC_O', 'NEOFAC_C', 'NEOFAC_N', 'NEOFAC_E']
edge_names = ['mean_FA', 'mean_strl', 'num_streamlines']
labels= ['Agreeableness', 'Openness', 'Conscientiousness', 'Neuroticism',
        'Extraversion']
mapping = {k: v for k, v in zip(labels, big5)}
mat = np.triu_indices(84)
mews = '/home/skapoor/Thesis/gmwcs-solver'
metrics = ['balanced_accuracy', 'accuracy', 'f1_weighted', 'roc_auc_ovr_weighted']
edges = [ 'pearson']
# note: right now the matrix whole is not scaled, for computing the fscores and correlation coeff it has to be so.
y_train = computed_subjects()
X_train = generate_combined_matrix(tri, list(y_train.index))  # need to check indices till here then convert to numpy array

#labels = ['Gender']
#mapping = {'Gender': 'Gender'}
y_test = test_subjects()
X_test = generate_test_data(tri, y_test.index)
#X_train = pd.concat([X_train, X_test],axis=0)
#y_train = pd.concat([y_train, y_test], axis=0)
num_strls = X_train.iloc[:, 2 * tri:]

mews = '/home/skapoor/Thesis/gmwcs-solver'
ip = BrainGraph('t_test', 'num_streamlines', 'const', 'Gender', 10, -0.01, 0)
feature_indices = ip.read_from_file(mews, False)
#ip.visualize_graph(mews, False, plotting_options=graph_options('red',10, 'yellow', 2, 1.5),
#                   figs=(10,6))

In [24]:
dict_lut = corresp_label_file('fs_default.txt')
mapping ={}
for edge in ip.edges:
    mapping[edge] = (dict_lut[edge[0]+1], dict_lut[edge[1]+1])
y_train = y_train['Gender']
m = y_train[y_train=='M']
f = y_train[y_train=='F']
l1 = []
for feature, edge in zip(feature_indices, mapping.values()):
    val = ttest_ind(num_strls.loc[m.index].iloc[:, feature], num_strls.loc[f.index].iloc[:, feature]).pvalue
    val = np.format_float_scientific(val, precision=3, exp_digits=2)
    l1.append([edge[0], edge[1], val])

In [25]:
l1 = pd.DataFrame(l1, columns=['ROI', 'ROI', 'p value'])
l1.to_csv('outputs/csvs/gender10_numstrls.csv')

In [ ]:
l1